In [ ]:
import sys
import random
import pickle
from os.path import join

sys.path.append('..')

import numpy as np
import networkx as nx

from cnrg.VRG import VRG as VRG
from cnrg.LightMultiGraph import convert
from cnrg.LightMultiGraph import LightMultiGraph as LightMultiGraph
from cnrg.generate import generate_graph
from cnrg.globals import find_boundary_edges
from cnrg.generate import generate_graph

import matplotlib.pyplot as plt
import matplotlib_inline
%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [ ]:
sys.path.append('../src')
from data import load_data
from decomposition import decompose, decompose_component
from adjoin_graph import update_grammar

# prelims

In [3]:
loaded = load_data(dataname='email-eucore', lookback=0)
graphs = [g for _, g in loaded]
times = [t for t, _ in loaded]

In [4]:
t = 13
g = graphs[t]
h = graphs[t + 1]

In [5]:
grammar = decompose(g, time=t)
grammar

100%|███████████████████████████████|[00:00<00:00]
100%|███████████████████████████████|[00:00<00:00]
100%|███████████████████████████████|[00:00<00:00]
100%|███████████████████████████████|[00:00<00:00]
100%|███████████████████████████████|[00:00<00:00]
100%|███████████████████████████████|[00:00<00:00]
100%|███████████████████████████████|[00:00<00:00]


graph: , mu: 4, type: mu_level_dl clustering: leiden rules: 245(245) mdl: 19_990.3 bits

In [6]:
igrammar = update_grammar(grammar, g, h, time=t+1, signature='i')
display(igrammar, igrammar.ll)

additions: 100%|██████████| 63/63 [00:00<00:00, 13781.94it/s]


graph: , mu: 4, type: mu_level_dl clustering: leiden rules: 245(245) mdl: 30_224.5 bits

-10.735853049472523

In [7]:
jagrammar = update_grammar(grammar, g, h, time=t+1, signature='ja')
display(jagrammar, jagrammar.ll)

additions: 100%|██████████| 11/11 [00:00<00:00, 15849.31it/s]


graph: , mu: 4, type: mu_level_dl clustering: leiden rules: 257(257) mdl: 30_742.3 bits

-10.678652412305714

In [8]:
jbgrammar = update_grammar(grammar, g, h, time=t+1, signature='jb')
display(jbgrammar, jbgrammar.ll)

additions: 100%|██████████| 11/11 [00:00<00:00, 14665.40it/s]


graph: , mu: 4, type: mu_level_dl clustering: leiden rules: 255(255) mdl: 30_581.8 bits

-10.674336240759926

# generation

In [9]:
base_n = g.order()
target_n = len(set(g.nodes()) | set(h.nodes()))
target_e = len(set(g.edges()) | set(h.edges()))
print(target_n, target_e, '|', g.order(), g.size(), '|', h.order(), h.size())

794 5285 | 620 1758 | 772 4243


In [10]:
base_g, base_ro = generate_graph(base_n, grammar, merge_iso_rules=False)
base_g

Generated graph: n = 641, m = 1781


n = 641 m = 1_781

In [11]:
ig, iro = generate_graph(target_n, igrammar, merge_iso_rules=False)
ig

Graph generated in 5 tries.
Generated graph: n = 805, m = 5513


n = 805 m = 5_513

In [12]:
jag, jaro = generate_graph(target_n, jagrammar, merge_iso_rules=False)
jag

Generated graph: n = 821, m = 5224


n = 821 m = 5_224

In [13]:
jbg, jbro = generate_graph(target_n, jbgrammar, merge_iso_rules=False)
jbg

Graph generated in 3 tries.
Generated graph: n = 775, m = 5162


n = 775 m = 5_162

In [ ]:
for rule, pidx, anode in jbgrammar:
    if rule.lhs != (ll := sum(d['b_deg'] for v, d in rule.graph.nodes(data=True))):
        print(rule.lhs, ll, rule, pidx, anode)

In [ ]:
for rule, _, _ in jbgrammar:
    try:
        if rule.branch:
            print(rule)
    except:
        continue

In [ ]:
jbgrammar.root